# 4.3 Variations on a Scheme - Nondeterministic Computing

# 4.3.1 Amb and Search

Introduce a new `amb` special form that allows nondeterministic computing by returning the value of *one* of a set of choices
- Short for 'ambiguous'

```scheme
(amb <e1> <e2> ... <en>)
```

Returns the value of *one* of the *n* expressions 'ambiguously'. For example, the following `list` expression can have *6* possible values:

```scheme
(list (amb 1 2 3) (amb 'a 'b))
;; (1 a) (1 b) (2 a) (2 b) (3 a) (3 b)
```

`amb` with a single choice -> ordinary value

`amb` with no no choices will "fail" the computation - abort and produce no value.

When a computation fails, the evaluator performs **depth-first search backtracking** - falling back to the most recent *choice* and attempting the next alternative.

## 4.35

In [ ]:
(define (an-integer-between low high)
  (require <= low high)
  (amb low (an-integer-between (+ low 1) high)))

(define (a-pythagorean-triple-between low high)
  (let ((i (an-integer-between low high)))
    (let ((j (an-integer-between i high)))
      (let ((k (an-integer-between j high)))
        (require (= (+ (* i i) (* j j)) 
                    (* k k)))
        (list i j k)))))

## 4.36

Redefining `a-pythagorean-triple-between` to use `an-integer-starting-from`:
```scheme
(define (a-pythagorean-triple-between low high)
  (let ((i (an-integer-starting-from low)))
    (let ((j (an-integer-starting-from i)))
      (let ((k (an-integer-starting-from j)))
        (require (= (+ (* i i) (* j j)) 
                    (* k k)))
        (list i j k)))))
```

`an-integer-starting-from` will *infinitely* generate integers greater than or equal to it's initial argument:

```scheme
(define (an-integer-starting-from n)
  (amb n (an-integer-starting-from (+ n 1))))
```

The `require` represents the constraint:

$i^2+j^2=k^2$

In `a-pythagorean-triple-between`, the initial values of `i`, `j` and `k`  are all equal to `low`. The `require` will therefore fail. The evaluator will then backtrack to attempt the next value of `k` which will again fail. As there is *no limit* on the value of `k` due to `an-integer-starting-from` infinitely producing values, the procedure will never terminate.

In order to generate all Pythagorean triples, the value of `k` must be bounded. Since `k` represents the length of the *hypotenuse*, we can bound `k` in relation to `i` and `j`:

$k > i, k > j$

Using the the [triangle inequality](https://en.wikipedia.org/wiki/Triangle_inequality), `k` can be further bounded:

$k <= i+j$

Combining these, `i` and `k` can be bounded using `an-integer-between` and `j` can use `an-integer-starting-from` with an initial value of `1`:

In [2]:
;; extracted out for clarity
(define (is-pythagorean-triple i j k)
  (= (+ (* i i) (* j j))
     (* k k)))

(define (a-pythagorean-triple)
  (let ((j (an-integer-starting-from 1)))
    (let ((i (an-integer-between 1 j))) ;; k > i AND k > j
      (let ((k (an-integer-between j (+ i j)))) ;; k <= i + j AND k > j
        (require (is-pythagorean-triple i j k))
        (list i j k)))))

## 4.37

Ben's solution is more efficient than the one in exercise 4.35. This is because it puts greater contstraints on the choices of values that must be tested, meaning many irrelevant results are ignored:

- `(require (>= hsq ksq)` will reject values for which the sum of the squares of `i` and `j` are greater than the square of `high` as these will definitely fail the pythagorean truple counstraint that $i^2+j^2=k^2$
- `(require (integer? k))` prevents testing for all values of `k` since the solution is found **only** when `k` is an integer.